In [ ]:
# Find out the size of vehicle

import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree

import sys

sys.path.append("/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance")
import constants

# Load the point cloud data for multiple frames
# cloud_point_data = np.load('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/17-10-2024/cloud_point_data2.npy', allow_pickle=True)[300:310]

import pickle
# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data4k.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)
    
print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[1300:1303]
print(f"Loaded {len(cloud_point_data)} point clouds")


# Parameters
num_frames = len(cloud_point_data)
eps = 0.5
min_samples = 10

# Cluster dimensions for your car
my_car_bbox_1 = [1.47, 1.26, 0.39]
my_car_bbox_2 = [1.68, 0.26, 0.07]

my_car_dis_1 = [1.3]
my_car_dis_2 = [2.3]


# Initialize tracking history (to store cluster IDs and their positions)
cluster_history = []
tracking_id = 0
cluster_color_map = {}  # Map to store consistent colors for clusters
previous_centroids = []  # Track centroids of clusters from the previous frame
previous_cluster_ids = []  # Track cluster IDs from the previous frame

def track_clusters(current_centroids, previous_centroids, prev_cluster_ids, threshold=1.0):
    """Match current centroids with previous centroids using nearest neighbor approach."""
    if len(previous_centroids) == 0:
        return np.arange(len(current_centroids)) + tracking_id, tracking_id + len(current_centroids)
    
    # Create a KDTree for fast nearest neighbor lookup
    tree = KDTree(previous_centroids)
    
    # Assign new tracking IDs based on nearest neighbors from the previous frame
    assigned_ids = np.full(len(current_centroids), -1)  # -1 for new clusters
    distances, indices = tree.query(current_centroids)
    
    # Assign tracking IDs based on distance threshold
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        if dist < threshold:
            assigned_ids[i] = prev_cluster_ids[idx]  # Match with existing cluster ID
    
    # Assign new IDs to unmatched clusters
    new_clusters = (assigned_ids == -1)
    assigned_ids[new_clusters] = np.arange(tracking_id, tracking_id + np.sum(new_clusters))
    
    return assigned_ids, tracking_id + np.sum(new_clusters)

def classify_vehicle(bbox):
    """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
    dimensions = bbox.get_extent() # Sort to get length, width, height in a consistent order
    dimensions = sorted(dimensions, reverse=True)
    print(dimensions)

    d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]

    for vehicle_type, size_range in constants.VEHICLE_SIZE_RANGES.items():
        if (size_range['0'][0] <= d0 <= size_range['0'][1] and
            size_range['1'][0] <= d1 <= size_range['1'][1] and
            size_range['2'][0] <= d2 <= size_range['2'][1]):
            return vehicle_type
    return None

def merge_clusters(clusters):
    """Merge two clusters into one."""
    merged_cluster = np.vstack(clusters)  # Stack them together
    return merged_cluster

def is_my_car(dimensions, centroid):
    """Check if the bounding box dimensions match either of the two known bounding boxes for your car."""
    dimensions = sorted(dimensions, reverse=True)
    if ((np.allclose(dimensions, my_car_bbox_1, atol=0.2) and (np.allclose(my_car_dis_1,[np.linalg.norm(centroid, axis=0)], atol=0.1))) or
        (np.allclose(dimensions, my_car_bbox_2, atol=0.2) and (np.allclose(my_car_dis_2,[np.linalg.norm(centroid, axis=0)], atol=0.1)))):
        return True
    return False


for frame_idx in range(num_frames):
    # DBSCAN clustering for the current frame
    current_frame = cloud_point_data[frame_idx]
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(current_frame)
    labels = clustering.labels_

    # Extract cluster centroids for the current frame
    current_centroids = []
    cluster_points = []
    
    for cluster_label in set(labels):
        if cluster_label == -1:
            continue  # Skip noise points
        
        cluster_data = current_frame[labels == cluster_label]
  
        if np.any(cluster_data[:, 2] >= 2):
            continue
        
        cluster_points.append(current_frame[labels == cluster_label])
        cluster_centroid = np.mean(current_frame[labels == cluster_label], axis=0)
        current_centroids.append(cluster_centroid)
    
    current_centroids = np.array(current_centroids)
    
    # Track clusters by matching current centroids with previous centroids
    if frame_idx == 0:
        # For the first frame, initialize the cluster IDs
        cluster_ids = np.arange(len(current_centroids))
        tracking_id = len(current_centroids)
    else:
        cluster_ids, tracking_id = track_clusters(current_centroids, previous_centroids, previous_cluster_ids)

    # Save the cluster history for visualization
    cluster_history.append((cluster_points, current_centroids, cluster_ids))
    
    # Update previous frame's centroids and IDs for the next iteration
    previous_centroids = current_centroids
    previous_cluster_ids = cluster_ids

    # Assign unique colors to clusters persistently across frames (tracking colors)
    for cluster_id in cluster_ids:
        if cluster_id not in cluster_color_map:
            color_index = len(cluster_color_map) % len(constants.COLOR_NAMES)  # Cycle through the color list
            cluster_color_map[cluster_id] = (constants.COLOR_NAMES[color_index], constants.COLOR_VALUES[constants.COLOR_NAMES[color_index]])

# Visualization across multiple frames with consistent colors for tracked clusters
for frame_idx, (cluster_points, centroids, cluster_ids) in enumerate(cluster_history):
    print(f"\nFrame {frame_idx}:")
    all_points = []
    all_colors = []
    bounding_boxes = []
    centroids_spheres = []
    
    for i, cluster in enumerate(cluster_points):
        # Add all points to the list for visualization
        all_points.append(cluster)
        
        # Create an Open3D point cloud for the current cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
        
        # Compute the bounding box for the cluster
        bbox = cluster_pcd.get_axis_aligned_bounding_box()
        dimensions = bbox.get_extent()
        dimension_text = f"0: {dimensions[0]:.2f}, 1: {dimensions[1]:.2f}, 2: {dimensions[2]:.2f}"
        
        # Get cluster color for tracking (original color for points)
        cluster_color_name, cluster_color_value = cluster_color_map[cluster_ids[i]]
        
        # Add color for visualization
        all_colors.append(np.tile(cluster_color_value, (len(cluster), 1)))  # Use tracking color for points
        
        # Add the bounding box for visualization
        bbox.color = (0, 1, 0)  # Set bounding box color to green
        bounding_boxes.append(bbox)
        
        # Use bounding box center as centroid and classify for vehicle types
        bbox_center = bbox.get_center()
        vehicle_type = classify_vehicle(bbox)
        
        if vehicle_type:
            centroid_color = constants.VEHICLE_CENTROID_COLORS[vehicle_type]  # Color based on vehicle type
            print(f" Cluster Color: {cluster_color_name}  Detected a {vehicle_type.upper()} with bounding box dimensions: {sorted(bbox.get_extent(), reverse=True)}")
        else:
            centroid_color = [1, 0, 0]  # Default red if no vehicle classification
            
        print(f"  Cluster Color: {cluster_color_name} --- Bounding Box Dimensions: {dimension_text}")
        
        # Create a sphere at the centroid with the appropriate vehicle type color
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
        sphere.translate(bbox_center)
        sphere.paint_uniform_color(centroid_color)  # Set centroid color based on vehicle type
        centroids_spheres.append(sphere)

    if len(all_points) > 0:
        # Flatten the points and colors arrays for visualization
        all_points = np.vstack(all_points)
        all_colors = np.vstack(all_colors)
        
        # Create an Open3D point cloud object for visualization
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(all_points)
        pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
        # Visualize all points with bounding boxes, tracked cluster colors, and red centroids
        o3d.visualization.draw_geometries([pcd] + bounding_boxes + centroids_spheres,
                                          zoom=0.4,
                                          front=[1.0, -1.0, 0.5],  # Set the side view of the camera
                                          lookat=[0.0, 0.0, 0.0],
                                          up=[0.0, 0.0, 1.0])



Loaded 2652 point clouds
Loaded 3 point clouds


In [45]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import pickle
import sys

sys.path.append("/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance")
import constants

# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)

print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[1300:1303]
print(f"Using {len(cloud_point_data)} point clouds for this analysis")

# Parameters
num_frames = len(cloud_point_data)
eps = 0.5
min_samples = 10

# Cluster dimensions for your car
my_car_bbox_1 = [1.47, 1.26, 0.39]
my_car_bbox_2 = [1.68, 0.26, 0.07]

my_car_dis_1 = [1.3]
my_car_dis_2 = [2.3]

def is_my_car(dimensions, centroid):
    """Check if the bounding box dimensions match either of the two known bounding boxes for your car."""
    dimensions = sorted(dimensions, reverse=True)
    if ((np.allclose(dimensions, my_car_bbox_1, atol=0.2) and (np.allclose(my_car_dis_1,[np.linalg.norm(centroid, axis=0)], atol=0.5))) or
        (np.allclose(dimensions, my_car_bbox_2, atol=0.2) and (np.allclose(my_car_dis_2,[np.linalg.norm(centroid, axis=0)], atol=0.5)))):
        return True
    return False

def classify_vehicle(bbox):
    """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
    dimensions = bbox.get_extent()  # Sort to get length, width, height in a consistent order
    dimensions = sorted(dimensions, reverse=True)

    d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]

    for vehicle_type, size_range in constants.VEHICLE_SIZE_RANGES.items():
        if (size_range['0'][0] <= d0 <= size_range['0'][1] and
            size_range['1'][0] <= d1 <= size_range['1'][1] and
            size_range['2'][0] <= d2 <= size_range['2'][1]):
            return vehicle_type
    return None

def merge_clusters(clusters):
    """Merge two clusters into one."""
    merged_cluster = np.vstack(clusters)  # Stack them together
    return merged_cluster

for frame_idx in range(num_frames):
    # DBSCAN clustering for the current frame
    current_frame = cloud_point_data[frame_idx]
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(current_frame)
    labels = clustering.labels_

    # Extract cluster centroids for the current frame
    current_centroids = []
    cluster_points = []
    bounding_boxes = []
    my_car_clusters = []
    
    cluster_centroids = {}

    for cluster_label in set(labels):
        if cluster_label == -1:
            continue  # Skip noise points
        
        cluster_data = current_frame[labels == cluster_label]
        if np.any(cluster_data[:, 2] >= 2):
            continue

        cluster_points.append(current_frame[labels == cluster_label])
        cluster_centroid = np.mean(current_frame[labels == cluster_label], axis=0)
        current_centroids.append(cluster_centroid)
        

        # Compute the bounding box for the cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster_data)
        bbox = cluster_pcd.get_axis_aligned_bounding_box()
        bounding_boxes.append(bbox)

        cluster_centroids[cluster_label] = [cluster_centroid, np.linalg.norm(cluster_centroid, axis=0), bbox.get_extent()]
        
        # Check if this cluster is part of your car
        if is_my_car(bbox.get_extent(), cluster_centroid):
            my_car_clusters.append(current_frame[labels == cluster_label])

    # If two clusters representing your car are detected, merge them
    if len(my_car_clusters) == 2:
        merged_car = merge_clusters(my_car_clusters)
        print(f"Frame {frame_idx}: Detected and merged 2 clusters as 'my car'.")
        
        # You can visualize or perform further processing on the merged cluster
        merged_car_pcd = o3d.geometry.PointCloud()
        merged_car_pcd.points = o3d.utility.Vector3dVector(merged_car)
        merged_car_bbox = merged_car_pcd.get_axis_aligned_bounding_box()
        print(f"Merged 'my car' Bounding Box Dimensions: {merged_car_bbox.get_extent()}")


Loaded 2652 point clouds
Using 3 point clouds for this analysis
[1.47022944688797, 1.2595897912979126, 0.3869032859802246]
1.3136295
[1.6654882431030273, 0.2577587366104126, 0.071036696434021]
2.3121388
Frame 0: Detected and merged 2 clusters as 'my car'.
Merged 'my car' Bounding Box Dimensions: [4.33812976 1.66548824 0.67268014]
[1.470234453678131, 1.259609341621399, 0.3866642117500305]
1.3322325
[1.68212890625, 0.257753849029541, 0.0707707405090332]
2.314504
Frame 1: Detected and merged 2 clusters as 'my car'.
Merged 'my car' Bounding Box Dimensions: [4.33813477 1.68212891 0.67238176]
[1.4700390696525574, 1.256933569908142, 0.3869571089744568]
1.333414
[1.68212890625, 0.24853515625, 0.06817805767059326]
2.3086534
Frame 2: Detected and merged 2 clusters as 'my car'.
Merged 'my car' Bounding Box Dimensions: [4.32872558 1.68212891 0.66982096]


In [23]:
a = np.array([1.47, 1.26, 0.39])
b = np.array([1.46, 1.25, 0.4])

# Check if 'a' and 'b' are close, allowing for a small difference (atol=0.01)
result = np.allclose(a, b, atol=1)

print(result)

True


In [ ]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import pickle
import sys

sys.path.append("/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance")
import constants

# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data4k.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)

print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[1300:1303]
print(f"Using {len(cloud_point_data)} point clouds for this analysis")

# Parameters
num_frames = len(cloud_point_data)
eps = 0.5
min_samples = 10

# Cluster dimensions for your car
my_car_bbox_1 = [1.47, 1.26, 0.39]
my_car_bbox_2 = [1.68, 0.26, 0.07]

# [1.47022944688797, 1.2595897912979126, 0.3869032859802246] 1.669863054635386
# [1.6654882431030273, 0.2577587366104126, 0.071036696434021] 2.3076453325458326

my_car_dis_1 = [1.6]
my_car_dis_2 = [2.3]

def is_my_car(dimensions, centroid):
    """Check if the bounding box dimensions match either of the two known bounding boxes for your car."""
    dimensions = sorted(dimensions, reverse=True)
    print(dimensions, np.linalg.norm(centroid, axis=0))
    if ((np.allclose(dimensions, my_car_bbox_1, atol=0.2) and (np.allclose(my_car_dis_1,[np.linalg.norm(centroid, axis=0)], atol=0.1))) or
        (np.allclose(dimensions, my_car_bbox_2, atol=0.2) and (np.allclose(my_car_dis_2,[np.linalg.norm(centroid, axis=0)], atol=0.1)))):
        return True
    return False

def classify_vehicle(bbox):
    """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
    dimensions = bbox.get_extent()  # Sort to get length, width, height in a consistent order
    dimensions = sorted(dimensions, reverse=True)

    d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]

    for vehicle_type, size_range in constants.VEHICLE_SIZE_RANGES.items():
        if (size_range['0'][0] <= d0 <= size_range['0'][1] and
            size_range['1'][0] <= d1 <= size_range['1'][1] and
            size_range['2'][0] <= d2 <= size_range['2'][1]):
            return vehicle_type
    return None

def merge_clusters(clusters):
    """Merge two clusters into one."""
    merged_cluster = np.vstack(clusters)  # Stack them together
    return merged_cluster

for frame_idx in range(num_frames):
    # DBSCAN clustering for the current frame
    current_frame = cloud_point_data[frame_idx]
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(current_frame)
    labels = clustering.labels_

    # Extract cluster centroids for the current frame
    current_centroids = []
    cluster_points = []
    bounding_boxes = []
    my_car_clusters = []
    
    cluster_centroids = {}

    for cluster_label in set(labels):
        if cluster_label == -1:
            continue  # Skip noise points
        
        cluster_data = current_frame[labels == cluster_label]
        if np.any(cluster_data[:, 2] >= 2):
            continue

        cluster_points.append(current_frame[labels == cluster_label])
        cluster_centroid = np.mean(current_frame[labels == cluster_label], axis=0)
        current_centroids.append(cluster_centroid)
        
        # Compute the bounding box for the cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster_data)
        bbox = cluster_pcd.get_axis_aligned_bounding_box()
        bounding_boxes.append(bbox)

        cluster_centroids[cluster_label] = [cluster_centroid, np.linalg.norm(cluster_centroid, axis=0), bbox.get_extent()]
        
        # Check if this cluster is part of your car
        if is_my_car(bbox.get_extent(), bbox.get_center()):
            my_car_clusters.append(current_frame[labels == cluster_label])

    # If two clusters representing your car are detected, merge them
    if len(my_car_clusters) == 2:
        merged_car = merge_clusters(my_car_clusters)
        print(f"Frame {frame_idx}: Detected and merged 2 clusters as 'my car'.")
        
        # Create a point cloud for the merged car
        merged_car_pcd = o3d.geometry.PointCloud()
        merged_car_pcd.points = o3d.utility.Vector3dVector(merged_car)
        merged_car_bbox = merged_car_pcd.get_axis_aligned_bounding_box()
        
        # Set the bounding box color for the car
        merged_car_bbox.color = (1, 0, 1)  # Set car bounding box color to red
        bounding_boxes.append(merged_car_bbox)
        
        print(f"Merged 'my car' Bounding Box Dimensions: {merged_car_bbox.get_extent()}")

    # Visualization across multiple frames with consistent colors for tracked clusters
    all_points = []
    all_colors = []
    centroids_spheres = []

    for i, cluster in enumerate(cluster_points):
        # Add all points to the list for visualization
        all_points.append(cluster)
        
        # Create an Open3D point cloud for the current cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
        
        # Compute the bounding box for the cluster
        bbox = cluster_pcd.get_axis_aligned_bounding_box()
        dimensions = bbox.get_extent()
        dimension_text = f"0: {dimensions[0]:.2f}, 1: {dimensions[1]:.2f}, 2: {dimensions[2]:.2f}"
        
        # Get cluster color for tracking (original color for points)
        cluster_color_name, cluster_color_value = constants.COLOR_VALUES['green'], [0, 1, 0]  # Default green color for tracking

        # Add color for visualization
        all_colors.append(np.tile(cluster_color_value, (len(cluster), 1)))  # Use tracking color for points
        
        # Use bounding box center as centroid and classify for vehicle types
        bbox_center = bbox.get_center()
        vehicle_type = classify_vehicle(bbox)
        
        if vehicle_type:
            centroid_color = constants.VEHICLE_CENTROID_COLORS[vehicle_type]  # Color based on vehicle type
            print(f" Cluster Color: {cluster_color_name}  Detected a {vehicle_type.upper()} with bounding box dimensions: {sorted(bbox.get_extent(), reverse=True)}")
        else:
            centroid_color = [1, 0, 0]  # Default red if no vehicle classification
            
        print(f"  Cluster Color: {cluster_color_name} --- Bounding Box Dimensions: {dimension_text}")
        
        # Create a sphere at the centroid with the appropriate vehicle type color
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
        sphere.translate(bbox_center)
        sphere.paint_uniform_color(centroid_color)  # Set centroid color based on vehicle type
        centroids_spheres.append(sphere)

    if len(all_points) > 0:
        # Flatten the points and colors arrays for visualization
        all_points = np.vstack(all_points)
        all_colors = np.vstack(all_colors)
        
        # Create an Open3D point cloud object for visualization
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(all_points)
        pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
        # Visualize all points with bounding boxes, including the car's bounding box, and centroids
        o3d.visualization.draw_geometries([pcd] + bounding_boxes + centroids_spheres,
                                          zoom=0.4,
                                          front=[1.0, -1.0, 0.5],  # Set the side view of the camera
                                          lookat=[0.0, 0.0, 0.0],
                                          up=[0.0, 0.0, 1.0])


Loaded 4625 point clouds
Using 3 point clouds for this analysis
[2.8732205033302307, 0.5226078033447266, 0.4938538074493408] 10.781328323832689
[24.213340759277344, 18.9471435546875, 3.788722515106201] 1.3128348434300352
[0.7645314931869507, 0.4105076789855957, 0.14805984497070312] 17.230530765086936
[0.8128318786621094, 0.7128839492797852, 0.5989933013916016] 20.932154251396653
[1.3308317065238953, 0.5892138481140137, 0.1776294708251953] 17.239109898288774
[2.0464271306991577, 1.5262203216552734, 1.314279556274414] 20.81097232984708
[1.4037275910377502, 0.8320102691650391, 0.03952598571777344] 15.77921585833793
[2.5177863240242004, 2.003159523010254, 1.6338109970092773] 16.52330319537623
[1.6565771102905273, 1.3051949888467789, 0.3010406494140625] 23.569951585555405
[0.8280924260616302, 0.7646293640136719, 0.06086158752441406] 18.67891235741644
[1.6425144672393799, 0.8919830918312073, 0.4352607727050781] 20.483074283473037
[0.7483349442481995, 0.3289155960083008, 0.02011045813560486] 